In [2]:
!pip install ultralytics realesrgan basicsr torch easyocr

  Using cached ultralytics-8.3.130-py3-none-any.whl.metadata (37 kB)
  Using cached realesrgan-0.3.0-py3-none-any.whl.metadata (17 kB)
  Using cached basicsr-1.4.2.tar.gz (172 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidi

In [1]:
import cv2
from paddleocr import PaddleOCR
from ultralytics import YOLO
import numpy as np
from utils import Utils
from sort.sort import Sort


/Users/Dima/Documents/dev/cv/license-plate-recognition/.venv/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


In [2]:
# init models
coco_model = YOLO("models/yolo11s.pt")
license_plate_model = YOLO("logs/retrain/weights/best.pt")
ocr = PaddleOCR(use_angle_cls=True, lang="en", use_gpu=False, show_log=False)

utils = Utils(license_plate_model, coco_model, ocr, None)

file_name = "video1.mp4"
video = cv2.VideoCapture(f"source/{file_name}")
fps = float(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
detecting_area = (width//4, height//4, width*3//4, height*3//4)

overlay = np.zeros((height, width, 3), dtype=np.uint8)
cv2.rectangle(overlay, (detecting_area[0], detecting_area[1]), (detecting_area[2], detecting_area[3]), (0, 255, 0), -1)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [3]:
# video writer setup
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(f"output/{file_name}", fourcc, fps, (width, height))

In [4]:
tracker = Sort(max_age=10, min_hits=3, iou_threshold=0.3)

tracked_objs = []
tracked_ids = []
license_plate_number = None

each_nth_frame = 1
frame_number = -1
while True:
    frame_number += 1
    ret, frame = video.read()
    
    # each 5th frame
    if (frame_number % each_nth_frame != 0):
        continue
    
    if not ret:
        break
    
    img = frame[detecting_area[1]:detecting_area[3], detecting_area[0]:detecting_area[2]]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    coco_results = coco_model.predict(img, conf=0.5)
    if not coco_results:
        continue
    
    detections = [
        [x1, y1, x2, y2, score]
        for x1, y1, x2, y2, score, cls in coco_results[0].boxes.data.tolist()
        if utils.is_vehicle(cls)
    ]
    
    detections = np.array(detections) if detections else np.empty((0, 5))
    tracked_objs = tracker.update(detections)
    
    new_ids = tracked_objs[:, -1]
    if set(new_ids) != set(tracked_ids) or len(tracked_ids) > 0 and license_plate_number is None:
        tracked_ids = new_ids
        print("Detecting IDs: ", tracked_ids)
        license_plate_number = None
        
        for vehicle in tracked_objs:
            print("Vehicle detected: ", vehicle)
            vehicle_img = utils.crop_vehicle(img, vehicle)
            
            license_plate_obj = utils.fetch_license_plate(vehicle_img)
            if license_plate_obj is not None:
                license_plate_img, x, y = license_plate_obj
                license_plate_number = utils.extract_license_plate_number(license_plate_img)
                
    utils.draw_license_plate_number(license_plate_number, frame, (0, 0))
    cv2.rectangle(frame, (detecting_area[0], detecting_area[1]), (detecting_area[2], detecting_area[3]), (0, 255, 0), 2)
    frame = cv2.addWeighted(overlay, 0.1, frame, 1, 0)
    out.write(frame)
    
out.release()
video.release()


0: 384x640 (no detections), 316.3ms
Speed: 7.1ms preprocess, 316.3ms inference, 14.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 294.9ms
Speed: 4.8ms preprocess, 294.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 198.6ms
Speed: 2.1ms preprocess, 198.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 222.7ms
Speed: 1.9ms preprocess, 222.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 307.4ms
Speed: 1.7ms preprocess, 307.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 213.1ms
Speed: 3.7ms preprocess, 213.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 200.8ms
Speed: 1.3ms preprocess, 200.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 198.0ms
Speed: 1.5ms prep